## Workflow plan

 - Create a list of SMARTS strings for the bond types reported in reference 10.1021/ed085p532 (Tables 2).
 - Develop an RDKit-based query tool that checks whether the current SDF file contains the molecules with specified bond type.
 - If a given bond type is found in the molecule, the constitutive correction for that species will be included.
 - Constitutive corrections are added along atomic Pascal constants (procedure for unknown molecule).

### Materials:
 - [Substructure Filtering in RDKit ](https://www.youtube.com/watch?v=Z1PrErlmTGI)
 - [SMARTS notation](https://en.wikipedia.org/wiki/SMILES_arbitrary_target_specification)

## Compounds loader

In [1]:
from pathlib import Path
from typing import Any

import pytest
from rdkit import Chem
from rdkit.Chem import Mol, MolToSmarts, RemoveHs

from src import DIAMAG_COMPOUND_CONSTITUTIVE_CORR_SUBDIR
from src.constants.common_molecules import COMMON_MOLECULES
from src.core.compound import MBCompound
from src.core.molecule import MBMolecule
from src.constants.bonds import DIAMAG_RELEVANT_BONDS
from src.loader import SDFLoader


### Constitutive corrections for relevent bond types

In [2]:
compound: MBCompound = SDFLoader.Load(
    "1,2,3-triazine.sdf", subdir=DIAMAG_COMPOUND_CONSTITUTIVE_CORR_SUBDIR
)

mol: MBMolecule = compound.GetMols(to_rdkit=False)[0]
print(f'SDF loaded first molecule SMARTS: {mol.smarts}')

match = False
for idx, diamag_relevant_bond in enumerate(DIAMAG_RELEVANT_BONDS):
    if mol.HasSubstructMatch(smarts=diamag_relevant_bond.SMARTS):
        match = True
        print(f'{idx}: Match: "{diamag_relevant_bond.formula}": {diamag_relevant_bond}')
    
if not match:
    print("No match found.")

SDF loaded first molecule SMARTS: [#7]1:[#7]:[#6]:[#6]:[#6]:[#7]:1
61: Match: "triazine": DiamagRelevantBond(sdf_file='1,2,3-triazine.sdf', constitutive_corr=-1.4, formula='triazine', SMARTS='[$(n1nnccc1),$(n1nccnc1),$(n1cncnc1),$([nH+]1nnccc1),$(n1[nH+]nccc1),$(n1n[nH+]ccc1),$([nH+]1nccnc1),$(n1[nH+]ccnc1),$(n1ncc[nH+]c1),$([nH+]1cncnc1),$(n1c[nH+]cnc1),$(n1cnc[nH+]c1)]', description="Assumes the same constant for three triazine isomers and thier monoprotonated states. This must be noted in Software's MANUAL.")


In [ ]:
from src import DIAMAG_COMPOUND_CONSTITUTIVE_CORR_SUBDIR
from src.constants.bonds import DIAMAG_RELEVANT_BONDS, DiamagRelevantBond
from src.core.compound import MBCompound
from src.loader import SDFLoader


def run_bond_match_tests_info_only() -> None:
    """
    Informational version:
    - No exceptions
    - Collects all failures
    - Prints a clean final summary
    """

    failures: list[dict] = []
    skipped: list[int] = []
    passed: list[int] = []

    for idx, drb in enumerate(DIAMAG_RELEVANT_BONDS):

        # --- Skip missing SDFs -----------------------------------------
        if not drb.sdf_file:
            print(
                f"[SKIPPED {idx}] Missing SDF file for: "
                f'"{drb.formula}"'
            )
            skipped.append(idx)
            continue

        # --- Load compound ---------------------------------------------
        compound: MBCompound = SDFLoader.Load(
            drb.sdf_file,
            subdir=DIAMAG_COMPOUND_CONSTITUTIVE_CORR_SUBDIR,
        )

        expected_smarts = drb.SMARTS
        test_failed = False

        # --- Check all molecules ---------------------------------------
        for mol in compound.GetMols(to_rdkit=False):
            actual_mol_sdf_smarts = mol.smarts

            if not mol.HasSubstructMatch(smarts=expected_smarts):
                test_failed = True

                failures.append(
                    {
                        "idx": idx,
                        "formula": drb.formula,
                        "sdf_file": drb.sdf_file,
                        "expected_smarts": expected_smarts,
                        "molecule_smarts": actual_mol_sdf_smarts,
                        "mol": mol,
                    }
                )

        if test_failed:
            print(f"[ERROR {idx}] {drb.formula}")
        else:
            # print(f"[OK {idx}] {drb.formula}")
            passed.append(idx)

    # --- Final summary --------------------------------------------------
    print("\n" + "=" * 80)
    print("SMARTS MATCH SUMMARY")
    print("=" * 80)

    print(f"✅ Passed : {len(passed)}")
    print(f"⚠️ Skipped: {len(skipped)}")
    print(f"❌ Failed : {len(failures)}")

    # --- Detailed failure report ---------------------------------------
    if failures:
        print("\n" + "-" * 80)
        print("DETAILED FAILURES")
        print("-" * 80)

        for f in failures:
            print(
                f"\n[Test {f['idx']} FAILED]\n"
                f"Bond formula:   {f['formula']}\n"
                f"SDF file:       {f['sdf_file']}\n\n"
                f"Expected SMARTS:\n"
                f"  {f['expected_smarts']}\n\n"
                f"Molecule SMARTS:\n"
                f"  {f['molecule_smarts']}\n\n"
                f"Molecule object:\n"
                f"  {f['mol']}\n"
                + "-" * 80
            )

    print("\n✅ Informational run completed.")


# --- Run in notebook or main ------------------------------------------
run_bond_match_tests_info_only()


[SKIPPED 4] Missing SDF file for: "CH2=CH-CH2-"
[SKIPPED 5] Missing SDF file for: "C=O"
[ERROR 11] -N#C
[ERROR 15] C-Cl
[ERROR 30] Ar-Ar
[ERROR 52] pyrazine

SMARTS MATCH SUMMARY
✅ Passed : 56
⚠️ Skipped: 2
❌ Failed : 4

--------------------------------------------------------------------------------
DETAILED FAILURES
--------------------------------------------------------------------------------

[Test 11 FAILED]
Bond formula:   -N#C
SDF file:       HCN-Any.sdf

Expected SMARTS:
  [N+;X2]#[C-;X1]

Molecule SMARTS:
  [#6]#[#7+]-[#6]

Molecule object:
  HCN-Any.sdf:0 (C#[N+]C)
--------------------------------------------------------------------------------

[Test 15 FAILED]
Bond formula:   C-Cl
SDF file:       R2CCl2.sdf

Expected SMARTS:
  [C;!$([C]([C])([C])([Cl])[Cl]);!$([C;H1]([C])([Cl])[Cl]);!$([C]([C])([C])([Cl])-[C]([C])([C])[Cl])]-[Cl]

Molecule SMARTS:
  [#6](-[#17])(-[#17])(-[#6])-[#6]

Molecule object:
  R2CCl2.sdf:0 (CC(C)(Cl)Cl)
--------------------------------------------